In [1]:
import re
from PyPDF2 import PdfReader
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

reader = PdfReader('phrase2.pdf')
text = ""
for page in reader.pages:
    text += page.extract_text() + "\n"

# text
tmp = text.split('\n')
tmp = [x.rstrip().lstrip() for x in tmp if re.search(r'^[0-9]', x) is not None]

In [2]:
# year_exam = [x.rstrip().lstrip() for x in tmp if any(y in x for y in ['學測', '指考'])]
year_exam = [x.split(' ',1) for x in tmp if re.search(r'^[0-9]+\.', x) is None]
phrase = [x.split(' ',1) for x in tmp if re.search(r'^[0-9]+\.', x) is not None]
phrase = [[i[0], i[1].replace('(1) ','@').replace(' (2) ', ';').replace(' (3) ', ';').replace(' (4) ', ';')] for i in phrase]

In [3]:
new_phrase = []
for i in phrase:
    if '@' in i[1]:
        i = [i[0], i[1].split('@')]
    else:
        delimiter = re.findall(r'[\u2E80-\u9FFF]', i[1])[0]
        i = [i[0], [x for x in i[1].split(delimiter, 1) if x]]
        i[1][1] = delimiter+i[1][1]
    new_phrase.append(i)

In [4]:
outer_list = []
inner_list = []
inner_list.append(new_phrase[0][1])
for i in new_phrase[1:]:
    if i[0]!='1.':
        inner_list.append(i[1])
    else:
        outer_list.append(inner_list)
        inner_list = []
        inner_list.append(i[1])
outer_list.append(inner_list)

In [5]:
df = pd.DataFrame({'year_exam' : year_exam,
                    'v1_phrase' : outer_list}, 
                    columns=['year_exam','v1_phrase'])
# df

In [6]:
df[['year', 'order']] = df['year_exam'].apply(pd.Series)
df = df.explode('v1_phrase')
df[['phrase', 'chinese']] = df['v1_phrase'].apply(pd.Series)
df.reset_index(drop=True, inplace=True)
# df

In [7]:
df.to_csv('phrase2.csv', encoding='utf-8-sig', columns=['order', 'year', 'phrase', 'chinese'])
tmp = pd.read_csv('phrase2.csv', index_col=0)
# pd.set_option('display.max_rows', None)
tmp

,order,year,phrase,chinese
0,指考,110,wear out,磨損 ; 耗盡
1,指考,110,break through,突破 ; 打破
2,指考,110,heat up,加熱 ; 使 ... 變暖
3,指考,110,in short,總⽽⾔之 ; 簡⽽⾔之
4,指考,109,serve as,充當 ; 起 … 的作⽤
...,...,...,...,...
223,學測,100,turn out,結果是
224,學測,100,appeal to,吸引
225,學測,100,be presented with,獲贈
226,學測,100,result in,造成 ; 導致
